# Install prerequsits

In [1]:

from IPython.display import clear_output

%pip install --upgrade pip
%pip install haystack-ai
%pip install pinecone-haystack
%pip install sentence-transformers
%pip install transformers[torch,sentencepiece]

clear_output()

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Initializaion

In [ ]:
import os

# Set the environment variable
os.environ["PINECONE_API_KEY"] = 'dda784f5-2d99-4031-931a-37bb9f08e100'

# Check if it's set
print(os.getenv("PINECONE_API_KEY"))


dda784f5-2d99-4031-931a-37bb9f08e100


In [ ]:
from haystack_integrations.document_stores.pinecone import PineconeDocumentStore

document_store = PineconeDocumentStore(
    index='mureq',
    environment='gcp-starter'
 )

# Preparing data and inserting





In [ ]:
import json
from sentence_transformers import SentenceTransformer
from haystack.document_stores.types import DuplicatePolicy
from haystack import Document
from haystack.components.embedders import SentenceTransformersTextEmbedder, SentenceTransformersDocumentEmbedder

In [ ]:
def convert(file_path):
        with open(file_path, 'r', encoding='utf-8')as f:
          data = json.load(f)

        extracted_data = []
        for item in data['data']:
            title = item.get('title', '')
            for paragraph in item.get('paragraphs', []):
                context = paragraph.get('context', '')
                extracted_data.append(Document(content=title+": "+context, meta={"title": title}))

        return extracted_data


extracted = convert('/content/CombinedData.json')

#مو لازم نشغلها كل مرة

In [ ]:
document_embedder = SentenceTransformersDocumentEmbedder(model="OmarAlsaabi/e5-base-mlqa-finetuned-arabic-for-rag")
document_embedder.warm_up()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.62k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

In [ ]:
documents_with_embeddings = document_embedder.run(extracted)
document_store.write_documents(documents_with_embeddings.get("documents"), policy=DuplicatePolicy.OVERWRITE)
#مو لازم نشغلها كل مرة

Batches:   0%|          | 0/15 [00:00<?, ?it/s]

Upserted vectors:   0%|          | 0/478 [00:00<?, ?it/s]

478

# Retriever

In [ ]:
from haystack_integrations.components.retrievers.pinecone import PineconeEmbeddingRetriever

retriever =  PineconeEmbeddingRetriever(document_store=document_store,top_k=2)


In [ ]:
from haystack import Pipeline

query_pipeline = Pipeline()
query_pipeline.add_component("text_embedder", SentenceTransformersTextEmbedder(model="OmarAlsaabi/e5-base-mlqa-finetuned-arabic-for-rag"))
query_pipeline.add_component("retriever",retriever)
query_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")


🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: PineconeEmbeddingRetriever
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])

# Reader

In [ ]:
from accelerate import Accelerator
from haystack.components.readers import ExtractiveReader

reader = ExtractiveReader(model='/content/drive/MyDrive/QAmodel_f181')
reader.warm_up()


In [ ]:
query_pipeline.add_component(instance=reader, name="reader")

# PIPELINE

In [ ]:
query_pipeline.connect("retriever.documents", "reader.documents")

🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: PineconeEmbeddingRetriever
  - reader: ExtractiveReader
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> reader.documents (List[Document])

In [ ]:
query = "ماهي طرق العناية بشجرةالسدر؟"
query_pipeline.run(data={"text_embedder": {"text": query},
                                   "reader": {"query": query, "top_k": 1}})

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'reader': {'answers': [ExtractedAnswer(query='ماهي طرق العناية بشجرةالسدر؟', score=0.7497811317443848, data='للعناية بشجرة السدر، تحتاج هذه الشجرة إلى ري وتقليم قليل و ويمكن أن تتحمل الملوحة المتوسطة ولكن المياه الراكدة لا تناسبها.', document=Document(id=29d7a7cd0d2b23618d4d013a218026d2af11e9864c2ff82be2e6d320fe7116ea, content: 'شجرة السدر -النبق-العبري (Zizyphus spina christi): للعناية بشجرة السدر، تحتاج هذه الشجرة إلى ري وتقل...', meta: {'title': 'شجرة السدر -النبق-العبري (Zizyphus spina christi)'}, score: 0.844797134, embedding: vector of size 768), context=None, document_offset=ExtractedAnswer.Span(start=51, end=173), context_offset=None, meta={}),
   ExtractedAnswer(query='ماهي طرق العناية بشجرةالسدر؟', score=0.25021886825561523, data=None, document=None, context=None, document_offset=None, context_offset=None, meta={})]}}

# GUI

In [ ]:
%pip install gradio
clear_output()

In [ ]:
def response(message, history):
      prediction = query_pipeline.run(data={"text_embedder": {"text": message},
                          "reader": {"query": message, "top_k": 1}})
      return prediction['reader']['answers'][0].data
      # if prediction['reader']['answers'][0].score > 0.2 :
      #    return prediction['reader']['answers'][0].data
      # else:
      #   return "نأسف مورق ليس لديه المعلومات الكافية عن استفسارك."



In [ ]:

css="""

.gradio-container{text-align:right;}
.message{width: auto;}
"""


In [ ]:
import gradio as gr


chatbot_interface = gr.ChatInterface(response,
                                      title='مُورِق',
                                      chatbot = gr.Chatbot(value =[ [None, "مرحبًا أنا مورق 🌳"], [None, "كيف أستطيع مساعدتك؟"] ]),
                                      textbox=gr.Textbox(placeholder=None,scale=7,rtl=True),
                                      submit_btn ="📨أرسل",
                                      undo_btn =  None,
                                      retry_btn = None,
                                      clear_btn = None,
                                      css=css,
                                      theme = 'abidlabs/Lime'
                                     )

chatbot_interface.launch()

themes/theme_schema@1.0.0.json:   0%|          | 0.00/14.2k [00:00<?, ?B/s]

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://d8fa4a8c56811809af.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
